In [3]:
!pip install opencv-python-headless


In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from PIL import Image

# --- Paths ---
img_dir =  r"C:\Users\Dhivya\Desktop\AIML\project\raw_images"
mask_dir = r"C:\Users\Dhivya\Desktop\AIML\project\labels"
excel_path = r"C:\Users\Dhivya\Desktop\AIML\project\Lifestages.xlsx"
out_dir = r"C:\Users\Dhivya\Desktop\AIML\project\YOLO21_11"
os.makedirs(out_dir, exist_ok=True)

# --- Read Excel (fixed to handle .xlsx) ---
if excel_path.endswith(".xlsx"):
    df = pd.read_excel(excel_path)
else:
    df = pd.read_csv(excel_path)

# --- Validate columns ---
required_cols = {"imageName", "stage", "center_x", "center_y"}
if not required_cols.issubset(df.columns):
    raise ValueError(f"❌ Missing columns in Excel. Required: {required_cols}")

# --- Map stage to class ID ---
stages = df["stage"].unique().tolist()
stage_to_id = {s: i for i, s in enumerate(stages)}
print("✅ Stage mapping:", stage_to_id)

# --- Function: Convert mask → bounding boxes ---
def mask_to_bbox(mask_path):
    """Return bounding boxes (x, y, w, h) for all connected components in a binary mask."""
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    if mask is None:
        print(f"⚠️ Could not read mask: {mask_path}")
        return []
    _, th = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return [cv2.boundingRect(cnt) for cnt in contours]

# --- Process each image ---
for img_name, group in df.groupby("imageName"):
    img_path = os.path.join(img_dir, img_name)
    mask_path = os.path.join(mask_dir, img_name.replace(".png", "_GT.png"))

    if not os.path.exists(img_path) or not os.path.exists(mask_path):
        print(f"⚠️ Missing image or mask for: {img_name}")
        continue

    bboxes = mask_to_bbox(mask_path)
    if len(bboxes) == 0:
        print(f"⚠️ No objects found in mask for {img_name}")
        continue

    with Image.open(img_path) as im:
        W, H = im.size

    txt_path = os.path.join(out_dir, img_name.replace(".png", ".txt"))
    with open(txt_path, "w") as f:
        for _, row in group.iterrows():
            stage = row["stage"]
            class_id = stage_to_id[stage]
            cx, cy = float(row["center_x"]), float(row["center_y"])

            # Find closest bounding box
            distances = [np.hypot((x + w/2) - cx, (y + h/2) - cy) for (x, y, w, h) in bboxes]
            idx = int(np.argmin(distances))
            x, y, w, h = bboxes[idx]

            # Convert to YOLO normalized format
            x_center = (x + w/2) / W
            y_center = (y + h/2) / H
            wn = w / W
            hn = h / H

            f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {wn:.6f} {hn:.6f}\n")

print("YOLO annotation files created for all images.")


C:\Users\Dhivya\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


✅ Stage mapping: {'Esch': 0, 'LT': 1, 'LR-ET': 2, 'Lsch': 3, 'MT': 4, 'R': 5, 'Seg': 6, 'Gam': 7, 'DEBRIS': 8, 'WBC': 9}
YOLO annotation files created for all images.


In [2]:
# --------------------------------------------------------------
# 0. Imports
# --------------------------------------------------------------
import shutil, random
from pathlib import Path
from sklearn.model_selection import train_test_split

# --------------------------------------------------------------
# 1. CONFIG – EDIT ONLY THESE TWO LINES
# --------------------------------------------------------------
# CHANGE THESE TO MATCH YOUR REAL FOLDERS
IMG_ORIG = Path(r'C:\Users\Dhivya\Desktop\AIML\project\raw_images')   
LBL_ORIG = Path(r'C:\Users\Dhivya\Desktop\AIML\project\YOLO')       

# Where to create the YOLO split
SPLIT_ROOT = Path(r'C:\Users\Dhivya\Desktop\AIML\project\yolo_split')

# --------------------------------------------------------------
# 2. CREATE YOLO SPLIT FOLDERS
# --------------------------------------------------------------
IMG_TRAIN = SPLIT_ROOT / 'images/train'
IMG_VAL   = SPLIT_ROOT / 'images/val'
IMG_TEST  = SPLIT_ROOT / 'images/test'

LBL_TRAIN = SPLIT_ROOT / 'labels/train'
LBL_VAL   = SPLIT_ROOT / 'labels/val'
LBL_TEST  = SPLIT_ROOT / 'labels/test'

for p in [IMG_TRAIN, IMG_VAL, IMG_TEST, LBL_TRAIN, LBL_VAL, LBL_TEST]:
    p.mkdir(parents=True, exist_ok=True)

print("Split folders ready")

Split folders ready


In [3]:
# Collect files
img_files = [f for f in IMG_ORIG.iterdir() if f.suffix.lower() in {'.png','.jpg','.jpeg'}]
lbl_files = [f for f in LBL_ORIG.iterdir() if f.suffix == '.txt']

img_stems = {f.stem for f in img_files}
lbl_stems = {f.stem for f in lbl_files}

assert img_stems == lbl_stems, "Image-label name mismatch!"
print(f"Found {len(img_files)} matched pairs")

# Split
random.seed(42)
train_imgs, temp = train_test_split(img_files, test_size=0.30, random_state=42)
val_imgs, test_imgs = train_test_split(temp, test_size=0.50, random_state=42)

print(f"Train: {len(train_imgs)} | Val: {len(val_imgs)} | Test: {len(test_imgs)}")

# Copy
def copy_pair(img_path, img_dir, lbl_dir):
    stem = img_path.stem
    shutil.copy(img_path, img_dir / img_path.name)
    shutil.copy(LBL_ORIG / f"{stem}.txt", lbl_dir / f"{stem}.txt")

for img in train_imgs: copy_pair(img, IMG_TRAIN, LBL_TRAIN)
for img in val_imgs:   copy_pair(img, IMG_VAL,   LBL_VAL)
for img in test_imgs:  copy_pair(img, IMG_TEST,  LBL_TEST)

print("Split complete!")

Found 883 matched pairs
Train: 618 | Val: 132 | Test: 133
Split complete!


In [4]:
yaml_content = f"""path: {SPLIT_ROOT}
train: images/train
val:   images/val
test:  images/test   # optional

names:
  0: Esch
  1: LT
  2: LR-ET
  3: Lsch
  4: MT
  5: R
  6: Seg
  7: Gam
  8: DEBRIS
  9: WBC
"""

yaml_path = SPLIT_ROOT / 'data.yaml'
yaml_path.write_text(yaml_content, encoding='utf-8')
print(f"data.yaml → {yaml_path}")

data.yaml → C:\Users\Dhivya\Desktop\AIML\project\yolo_split\data.yaml


In [5]:
import random
sample = random.choice(list(LBL_TRAIN.glob('*.txt')))
print(f"Sample label: {sample.name}")
print(sample.read_text().splitlines()[:2])

Sample label: Trip 038 Day 1 01-12-05 Image 10 add_1.txt
['0 0.353111 0.269417 0.062229 0.090291', '2 0.425832 0.480097 0.060058 0.076699']


In [6]:
from ultralytics import YOLO
model = YOLO('yolov8s.pt')
model.train(
    data=str(yaml_path),
    epochs=10,
    imgsz=640,
    batch=16,
    name='malaria_v1',
    project='runs'
)

New https://pypi.org/project/ultralytics/8.3.230 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.228  Python-3.13.5 torch-2.8.0+cpu CPU (Intel Core Ultra 7 155U)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\Dhivya\Desktop\AIML\project\yolo_split\data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=malaria_v12

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 8, 9])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001CEC1B28050>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.

In [1]:
import os
import cv2

IMG_DIR = r"C:\Users\Dhivya\Desktop\AIML\project\raw_images"
LBL_DIR = r"C:\Users\Dhivya\Desktop\AIML\project\YOLO21_11"
OUT_DIR = r"C:\Users\Dhivya\Desktop\AIML\project\resnet_dataset"

os.makedirs(OUT_DIR, exist_ok=True)

for txt_file in os.listdir(LBL_DIR):
    if not txt_file.endswith(".txt"):
        continue

    stem = txt_file.replace(".txt", "")
    img_path = os.path.join(IMG_DIR, stem + ".png")
    label_path = os.path.join(LBL_DIR, txt_file)

    img = cv2.imread(img_path)
    H, W = img.shape[:2]

    with open(label_path, "r") as f:
        for i, line in enumerate(f):
            cls, xc, yc, w, h = map(float, line.split())

            # YOLO normalized → pixel coordinates
            x = int((xc - w/2) * W)
            y = int((yc - h/2) * H)
            w = int(w * W)
            h = int(h * H)

            crop = img[y:y+h, x:x+w]
            class_folder = os.path.join(OUT_DIR, str(int(cls)))
            os.makedirs(class_folder, exist_ok=True)

            cv2.imwrite(os.path.join(class_folder, f"{stem}_{i}.png"), crop)

print("Crops created for ResNet classification dataset!")


Crops created for ResNet classification dataset!
